# Unifying our datasets

This file was last used on Oct. 23 at 2:30 am. Adel is currently out of wits.

Commentary will be provided at a later date.

In [289]:
import pandas as pd
import numpy as np
import json

In [294]:
maps = pd.read_json('esports-data/mapping_data.json')
teams_maps_df= maps['teamMapping'].apply(pd.Series)
teams_maps_df.head()

teams_maps_df.rename(columns={'200':'red','100':'blue'},inplace=True)
teams_maps_df

,red,blue
0,105550026570060790,105550033967461806
1,105550005698683818,109485335453835911
2,108352305932141947,109696092218588987
3,99566406332987990,107700204561086446
4,105550001032913831,109485335453835911
...,...,...
25278,105515345386578249,103935468920814040
25279,108573042084687884,107251231972989211
25280,99322214691747156,101383793569248484
25281,100205573496804586,100205573495116443


In [295]:
maps.head()

,esportsGameId,platformGameId,teamMapping,participantMapping
0,110378429158160384,ESPORTSTMNT01:3416295,"{'200': '105550026570060790', '100': '10555003...","{'3': '107492116585043595', '5': '105388980252..."
1,110471139171602608,ESPORTSTMNT02:3228341,"{'200': '105550005698683818', '100': '10948533...","{'3': '107560281476330464', '5': '107559633217..."
2,110535609417029696,ESPORTSTMNT04:2693199,"{'200': '108352305932141947', '100': '10969609...","{'3': '105536902415377888', '5': '109696579630..."
3,110847390580148352,ESPORTSTMNT01:3413275,"{'200': '99566406332987990', '100': '107700204...","{'3': '98767991808793901', '5': '1096429488383..."
4,110413246204026224,ESPORTSTMNT01:3412587,"{'200': '105550001032913831', '100': '10948533...","{'3': '107560281476330464', '5': '107559633217..."


## Piece together the dataset after obtaining scores from Sarah's algorithm

In [296]:
with open("esports-data/mapping_data.json", "r") as json_file:
   mappings_data = json.load(json_file)

#For the sake of processing our events one by one, effortlessly, it's better to have our earliest tournaments processed first.
#The Elo rating file will go row by row, from first to last.
tour_df = pd.read_json("esports-data/tournaments-cleaned.json")
tour_df.sort_values(by='startDate',ascending=True,inplace=True)
tour_df.to_json('revamped-tourneys.json',orient='records')
tour_df.head()

,id,leagueId,name,slug,sport,startDate,endDate,stages,leagueName
197,103462459318635408,98767991302996016,LEC Spring 2020,lec_spring_2020,lol,2020-01-24,2020-04-27,"[{'name': 'Regular Season', 'type': None, 'slu...",LEC
196,103462439438682784,98767991299243168,LCS Spring 2020,lcs_spring_2020,lol,2020-01-24,2020-04-20,"[{'name': 'Regular Season', 'type': None, 'slu...",LCS
195,103478354329449184,98767991332355504,CBLOL 2020 Split 1,cblol_2020_split1,lol,2020-01-25,2020-05-03,"[{'name': 'Regular Season', 'type': None, 'slu...",CBLOL
194,103535401218775280,98767991331560960,OPL 2020 Split 1,opl_2020_split1,lol,2020-01-31,2020-04-24,"[{'name': 'Regular Season', 'type': None, 'slu...",LCO
193,103495775740097552,98767991343597632,TCL Winter 2020,tcl_winter_2020,lol,2020-02-01,2020-04-25,"[{'name': 'Regular Season', 'type': None, 'slu...",TCL


In [ ]:
with open("revamped-tourneys.json", "r") as json_file:
   tournaments_data = json.load(json_file)

In [297]:

#dict_game_logs:
games_riot_data = []

mappings = {
   esports_game["esportsGameId"]: esports_game for esports_game in mappings_data
}

#Unused, was there to cross-check data
playing_teams = {
   esports_game["esportsGameId"]: {'blue':esports_game["teamMapping"].get("100",None),
                                  'red':esports_game["teamMapping"].get("200",None)} for esports_game in mappings_data
}

game_counter = 0

#This is where it goes a bit bleh
for tournament in tournaments_data:
    start_date = tournament.get("startDate", "")
    #Tweaking Riot's code here: I might want to simply download the entire dataset.
    correct_year = True

    if correct_year:
        
        for stage in tournament["stages"]:
            for section in stage["sections"]:
                for match in section["matches"]:
                    for game in match["games"]:
                        if game["state"] == "completed":
                            game_id = game.get('id',None)
                            platform_game_id = mappings.get(game_id,{}).get("platformGameId",None)
                            game_leaguetag = tournament.get('leagueName',None)
                            game_leagueid = tournament.get('leagueId',None)
                            game_leaguestage = tournament.get('name',None)
                            game_toursection = section.get('name',None)
                            blue_team = None
                            red_team = None
                            winner = None
                            for team in game["teams"]:
                                blue_team = team.get('id') if team.get('side',None)=='blue' else blue_team
                                red_team = team.get('id') if team.get('side',None)=='red' else red_team
                                winner = team.get('side',None) if team.get('result',{}).get('outcome') == 'win' else winner
                            game_num = game.get('number',1)
                            tourney_id = tournament.get('id',None)

                            #Unused:
                            blue_mapping = playing_teams.get(game_id,{}).get("blue",None)
                            red_mapping = playing_teams.get(game_id,{}).get("red",None)

                            games_riot_data.append({
                                'leagueId': game_leagueid,
                                'leagueLabel': game_leaguetag,
                                'year':start_date[:4],
                                'stageTournament':game_leaguestage,
                                'stageRound':game_toursection,
                                'gameId':game_id,
                                'gameNumber':game_num,
                                'platformGameId':platform_game_id,
                                'blue':blue_team,
                                'red':red_team,
                                'winner':winner
                            })
                            
games_df = pd.DataFrame(games_riot_data)
                            

In [298]:
games_df

,leagueId,leagueLabel,year,stageTournament,stageRound,gameId,gameNumber,platformGameId,blue,red,winner
0,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524805,1,ESPORTSTMNT05:1410760,101383793569248484,101383793567806688,blue
1,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524807,1,ESPORTSTMNT05:1410131,101383793572656373,101383793567806688,blue
2,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524809,1,ESPORTSTMNT05:1400210,101383793574360315,101383793567806688,blue
3,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524811,1,ESPORTSTMNT04:1120607,103461966965149786,101383793567806688,blue
4,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524813,1,ESPORTSTMNT04:1130210,101383793567806688,98767991866488695,red
...,...,...,...,...,...,...,...,...,...,...,...
20529,98767991302996016,LEC,2023,Season Finals 2023,Regional Finals,110848560875705971,4,ESPORTSTMNT05:3241582,98767991866488695,101383793572656373,blue
20530,98767991302996016,LEC,2023,Season Finals 2023,Regional Finals,110848560875705972,5,ESPORTSTMNT05:3239537,101383793572656373,98767991866488695,red
20531,98767991302996016,LEC,2023,Season Finals 2023,Regional Finals,110848560875705974,1,ESPORTSTMNT05:3241555,107563714667537640,101383793567806688,blue
20532,98767991302996016,LEC,2023,Season Finals 2023,Regional Finals,110848560875705975,2,ESPORTSTMNT05:3240542,101383793567806688,107563714667537640,red


### Cleaning up some of our variables

In [299]:
games_df.loc[games_df.stageRound=='regional_qualifier','stageRound'] = 'Playoffs'
games_df.stageRound.unique()

array(['Round Robin', 'Playoffs', 'promotion', 'Group A', 'Group B',
       'Group C', 'Group D', 'play_in_knockouts', 'Play In Knockouts',
       'All-Star Exhibition', 'bracket', 'Regular Season', 'round_1',
       'round_2', 'knockouts', 'Groups', 'Play In Groups',
       'Regional Finals', 'weekend_5'], dtype=object)

In [300]:
games_df.drop(games_df[games_df['stageRound']=='All-Star Exhibition'].index,inplace=True)
games_df.stageRound.unique()

array(['Round Robin', 'Playoffs', 'promotion', 'Group A', 'Group B',
       'Group C', 'Group D', 'play_in_knockouts', 'Play In Knockouts',
       'bracket', 'Regular Season', 'round_1', 'round_2', 'knockouts',
       'Groups', 'Play In Groups', 'Regional Finals', 'weekend_5'],
      dtype=object)

In [301]:
games_df.loc[games_df.stageRound=='play_in_knockouts','stageRound'] = 'Play In Knockouts'
games_df.stageRound.unique()

array(['Round Robin', 'Playoffs', 'promotion', 'Group A', 'Group B',
       'Group C', 'Group D', 'Play In Knockouts', 'bracket',
       'Regular Season', 'round_1', 'round_2', 'knockouts', 'Groups',
       'Play In Groups', 'Regional Finals', 'weekend_5'], dtype=object)

In [302]:
games_df[games_df.stageRound=='weekend_5']

,leagueId,leagueLabel,year,stageTournament,stageRound,gameId,gameNumber,platformGameId,blue,red,winner
15856,98767991349978720,LJL,2023,Spring 2023,weekend_5,109710208773727577,1,ESPORTSTMNT01:3311922,101428372598668846,101428372607057464,blue
15857,98767991349978720,LJL,2023,Spring 2023,weekend_5,109710208773727578,2,ESPORTSTMNT01:3312009,101428372607057464,101428372598668846,blue
15858,98767991349978720,LJL,2023,Spring 2023,weekend_5,109710208773727579,3,ESPORTSTMNT01:3310972,101428372598668846,101428372607057464,blue
15859,98767991349978720,LJL,2023,Spring 2023,weekend_5,109710208773727581,1,ESPORTSTMNT02:3163537,101428372607057464,101428372603715124,red
15860,98767991349978720,LJL,2023,Spring 2023,weekend_5,109710208773727582,2,ESPORTSTMNT02:3163551,101428372607057464,101428372603715124,blue
...,...,...,...,...,...,...,...,...,...,...,...
19224,98767991349978720,LJL,2023,Summer 2023,weekend_5,110395308895739939,1,ESPORTSTMNT01:3376241,103535282119620510,101428372602011186,blue
19225,98767991349978720,LJL,2023,Summer 2023,weekend_5,110395308895739940,2,ESPORTSTMNT01:3375277,101428372602011186,103535282119620510,red
19226,98767991349978720,LJL,2023,Summer 2023,weekend_5,110395308895739943,1,ESPORTSTMNT01:3375747,101428372602011186,101428372605353526,blue
19227,98767991349978720,LJL,2023,Summer 2023,weekend_5,110395308895739944,2,ESPORTSTMNT01:3378654,101428372605353526,101428372602011186,blue


In [303]:
games_df.loc[games_df.stageRound=='round_1','stageRound']='Regular Season'

In [304]:
games_df.loc[games_df.stageRound=='round_2','stageRound']='Regular Season'

In [305]:
games_df.loc[games_df.stageRound=='Group A','stageRound']='Group Stage'

In [306]:
games_df.loc[games_df.stageRound=='Group B','stageRound']='Group Stage'

In [307]:
games_df.loc[games_df.stageRound=='Group C','stageRound']='Group Stage'

In [308]:
games_df.loc[games_df.stageRound=='Group D','stageRound']='Group Stage'

In [309]:
games_df.loc[games_df.stageRound=='promotion','stageRound']='Promotion Qualifiers'

In [310]:
games_df.loc[games_df.stageRound=='bracket','stageRound']='Playoffs'

In [311]:
games_df.loc[games_df.stageRound=='knockouts','stageRound']='Playoffs'

In [312]:
games_df.loc[games_df.stageRound=='Groups','stageRound']='Group Stage'

In [313]:
games_df.loc[games_df.stageRound=='weekend_5','stageRound']='Regular Season'

In [314]:
games_df.stageRound.unique()

array(['Round Robin', 'Playoffs', 'Promotion Qualifiers', 'Group Stage',
       'Play In Knockouts', 'Regular Season', 'Play In Groups',
       'Regional Finals'], dtype=object)

In [315]:
games_df.stageTournament.unique()

array(['LEC Spring 2020', 'LCS Spring 2020', 'CBLOL 2020 Split 1',
       'OPL 2020 Split 1', 'TCL Winter 2020', 'LCK 2020 Spring Split',
       'LJL Spring 2020', 'LLA 2020 Spring Split',
       'european-masters-spring-2020', 'opl_split2_2020',
       'TCL Summer 2020', 'CBLOL Split 2 2020', 'LCS Summer 2020',
       'LEC Summer 2020', 'LJL Summer 2020', 'LCK Summer 2020',
       'lla-closing-2020', 'PCS Summer 2020', 'eu_masters_summer_2020',
       'worlds_2020', 'LEC Spring 2021',
       'Greek Legends League Spring 2021', 'LCK Spring 2021',
       'PG Nationals Spring 2021', 'LCS Lock In 2021',
       'CBLOL Split 1 2021', 'LCK Challengers Spring 2021',
       'Superliga Spring 2021', 'Ultraliga Spring 2021',
       'nlc_2021_split1', 'CBLOL Academy Split 1 2021', 'lfl_2021_spring',
       'Prime League Spring 2021', 'TCL Winter 2021', 'LJL Spring 2021',
       'Hitpoint Masters Spring 2021', 'Liga Portuguesa Spring 2021',
       'Esports Balkan League Spring 2021', 'LLA Opening 

In [316]:
games_df.loc[games_df.stageTournament=='european-masters-spring-2020','stageTournament']='EU Masters Spring 2020'

In [317]:
games_df.loc[games_df.stageTournament=='eu_masters_summer_2020','stageTournament']='EU Masters Summer 2020'

In [318]:
games_df.loc[games_df.stageTournament=='opl_split2_2020','stageTournament']='OPL 2020 Split 2'

In [319]:
games_df.loc[games_df.stageTournament=='lla-closing-2020','stageTournament']='LLA Closing 2020'

In [320]:
games_df.loc[games_df.stageTournament=='worlds_2020','stageTournament']='Worlds 2020'

In [321]:
games_df.loc[games_df.stageTournament=='nlc_2021_split1','stageTournament']='NLC Spring 2021'

In [322]:
games_df.loc[games_df.stageTournament=='lfl_2021_spring','stageTournament']='LFL Spring 2021'

In [323]:
games_df.loc[games_df.stageTournament=='mss 2021 (archive)','stageTournament']='2021 Mid-Season Showdown'

In [324]:
games_df.loc[games_df.stageTournament=='european_masters_spring_2021_play_ins','stageTournament']='EU Masters 2021 Spring Play Ins'

In [325]:
games_df.loc[games_df.stageTournament=='EU Masters 2021 Spring Play Ins','stageTournament']='EU Masters Spring 2021 - Play Ins'

In [326]:
games_df.loc[games_df.stageTournament=='european_masters_spring_2021_main_event','stageTournament']='EU Masters Spring 2021'

In [327]:
games_df.loc[games_df.stageTournament=='msi_2021','stageTournament']='MSI 2021'

In [328]:
games_df.loc[games_df.stageTournament=='european_masters_summer_2021_main_event','stageTournament']='EU Masters Summer 2021'

### Import Sarah's scores and apply them to our dataframe, while recovering game dates

In [ ]:
scores_df = pd.read_csv('game_scores.csv',sep=';')
scores_df

In [330]:
dates = pd.read_csv('hackathon-riot-data.csv',sep=';')
dates.head()

,gameDate,esportsPlatformId,gameVersion,gameDuration,NbWardsPlacedBlue,NbWardsPlacedRed,NbControlWardsPlacedBlue,NbControlWardsPlacedRed,NbWardsKilledBlue,NbWardsKilledRed,...,TotalCCDurationMidBlue,TotalCCDurationADBlue,TotalCCDurationSupBlue,TotalCCDurationTopRed,TotalCCDurationJgRed,TotalCCDurationMidRed,TotalCCDurationADRed,TotalCCDurationSupRed,winner,DragonSoulTeam
0,2023-06-18 18:14:11.781000+00:00,ESPORTSTMNT01:3379581,13.11.512.8126,2068.268,54,58,15,23,9,12,...,14.136205,1.084917,23.350250,10.321404,39.145645,14.924883,9.256816,24.410091,red,NaN
1,2023-02-16 01:59:23.961000+00:00,ESPORTSTMNT02:3150844,13.3.491.6222,1999.819,0,0,0,0,19,37,...,16.207073,2.589555,30.072721,26.208740,61.781170,24.421112,10.136894,35.901794,red,red
2,2021-03-13 21:07:56.263000+00:00,ESPORTSTMNT02:1721741,11.5.361.5971,1882.763,77,143,36,37,38,29,...,8.929164,10.972918,28.091301,22.643715,4.892192,17.852480,0.000000,31.759960,blue,NaN
3,2021-04-10 11:56:11.980000+00:00,ESPORTSTMNT06:1680685,11.6.364.6814,1551.344,42,56,23,17,23,13,...,22.140129,2.105497,15.038666,14.537760,17.877005,19.403763,5.604137,15.086430,blue,NaN
4,2023-06-08 19:47:14.554000+00:00,ESPORTSTMNT06:2815912,13.10.509.8402,2246.940,71,70,38,36,22,26,...,14.882497,3.374509,25.278595,15.675308,20.266357,41.362591,7.650427,18.093018,red,NaN


In [331]:
dates = dates[['esportsPlatformId','gameDate']].copy()

In [332]:
scores_df = scores_df.merge(right=dates,how='left',on='esportsPlatformId')

In [333]:
scores_df

,esportsPlatformId,gameScore,gameDate
0,ESPORTSTMNT01:3379581,7.698220,2023-06-18 18:14:11.781000+00:00
1,ESPORTSTMNT02:3150844,12.364273,2023-02-16 01:59:23.961000+00:00
2,ESPORTSTMNT02:1721741,-9.231462,2021-03-13 21:07:56.263000+00:00
3,ESPORTSTMNT06:1680685,-9.784128,2021-04-10 11:56:11.980000+00:00
4,ESPORTSTMNT06:2815912,7.207281,2023-06-08 19:47:14.554000+00:00
...,...,...,...
19733,ESPORTSTMNT04:2689259,-0.718069,2023-07-26 15:59:21.957000+00:00
19734,ESPORTSTMNT05:3090546,-11.338861,2022-08-30 16:35:42.548000+00:00
19735,ESPORTSTMNT01:3343374,10.956199,2023-04-11 23:15:16+00:00
19736,ESPORTSTMNT03:3187730,12.134821,2023-07-15 19:10:14.307000+00:00


In [334]:
games_df.rename(columns={'platformGameId':'esportsPlatformId'},inplace=True)

In [335]:
games_df.head()

,leagueId,leagueLabel,year,stageTournament,stageRound,gameId,gameNumber,esportsPlatformId,blue,red,winner
0,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524805,1,ESPORTSTMNT05:1410760,101383793569248484,101383793567806688,blue
1,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524807,1,ESPORTSTMNT05:1410131,101383793572656373,101383793567806688,blue
2,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524809,1,ESPORTSTMNT05:1400210,101383793574360315,101383793567806688,blue
3,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524811,1,ESPORTSTMNT04:1120607,103461966965149786,101383793567806688,blue
4,98767991302996016,LEC,2020,LEC Spring 2020,Round Robin,103462460596524813,1,ESPORTSTMNT04:1130210,101383793567806688,98767991866488695,red


In [336]:
games_df = games_df.merge(right=scores_df,how='left',on='esportsPlatformId')

In [337]:
games_df.gameScore.isnull().sum()

796

Some games don't have data, no matter how we try to slice it. So, I've taken the easy out by giving blue teams the median score for a Blue victory, and red team wins the median score for a red team victory.

In [338]:
games_df.loc[(games_df.gameScore.isnull()) & (games_df.winner=='blue'),'gameScore']=-12 #Set to median for wins on blue side.
games_df.loc[(games_df.gameScore.isnull()) & (games_df.winner=='red'),'gameScore']=12 #Set to median for wins on red side.

In [339]:
games_df.gameScore.isnull().sum()

0

In [341]:
games_df.loc[games_df.stageRound=='Round Robin','stageRound']='Regular Season'

In [342]:
games_df[games_df.stageTournament=='Split 1 2023']

,leagueId,leagueLabel,year,stageTournament,stageRound,gameId,gameNumber,esportsPlatformId,blue,red,winner,gameScore,gameDate
14385,105709090213554608,LCO,2023,Split 1 2023,Regular Season,109675619696367989,1,ESPORTSTMNT03:3091057,98767991921462763,109675484017801229,blue,-6.361771,2023-01-24 09:09:56.858000+00:00
14386,105709090213554608,LCO,2023,Split 1 2023,Regular Season,109675619696367991,1,ESPORTSTMNT03:3096688,109675484017801229,109675490370327425,blue,-9.886028,2023-02-01 08:06:49.262000+00:00
14387,105709090213554608,LCO,2023,Split 1 2023,Regular Season,109675619696367993,1,ESPORTSTMNT03:3086775,109675484017801229,99566406332987990,red,10.289712,2023-01-23 07:15:07.756000+00:00
14388,105709090213554608,LCO,2023,Split 1 2023,Regular Season,109675619696367995,1,None,105709099258505657,109675484017801229,red,12.000000,NaN
14389,105709090213554608,LCO,2023,Split 1 2023,Regular Season,109675619696367997,1,ESPORTSTMNT01:3301927,101383792887446028,109675484017801229,red,13.711137,2023-02-06 07:07:22.874000+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15423,105549980953490848,CBLOL Academy,2023,Split 1 2023,Playoffs,110064346374946306,1,ESPORTSTMNT04:2672638,109485335453835911,109485351596724390,red,9.978496,2023-04-22 16:26:13.556000+00:00
15424,105549980953490848,CBLOL Academy,2023,Split 1 2023,Playoffs,110064346374946307,2,ESPORTSTMNT04:2672640,109485351596724390,109485335453835911,blue,-5.358027,2023-04-22 17:24:37.747000+00:00
15425,105549980953490848,CBLOL Academy,2023,Split 1 2023,Playoffs,110064346374946308,3,ESPORTSTMNT04:2672641,109485335453835911,109485351596724390,blue,-15.046228,2023-04-22 18:17:46.747000+00:00
15426,105549980953490848,CBLOL Academy,2023,Split 1 2023,Playoffs,110064346374946309,4,ESPORTSTMNT04:2670665,109485351596724390,109485335453835911,red,12.945521,2023-04-22 19:06:26.225000+00:00


In [343]:
games_df.leagueLabel.unique()

array(['LEC', 'LCS', 'CBLOL', 'LCO', 'TCL', 'LCK', 'LJL', 'LLA',
       'EMEA Masters', 'PCS', 'Worlds', 'Greek Legends League',
       'PG Nationals', 'LCK Challengers', 'SuperLiga', 'Ultraliga', 'NLC',
       'CBLOL Academy', 'La Ligue Française', 'Prime League',
       'Hitpoint Masters', 'Liga Portuguesa', 'Esports Balkan League',
       'LCL', 'MSI', 'LJL Academy', 'VCS', 'Elite Series',
       'College Championship', 'LCK Academy',
       'LCS Challengers Qualifiers', 'LCS Challengers', 'Arabian League',
       'North Regional League', 'LPL', 'South Regional League'],
      dtype=object)

That said, our games need a date/time at which they took place. I don't need a factual date, just a value that will allow me to go through games in the sequence in which they were introduced.

In [340]:
games_df.gameDate.isnull().sum()

796

In [345]:
games_df.gameDate = pd.to_datetime(games_df.gameDate,format='mixed')

In [346]:
import datetime

In [347]:
def timestamper(d):
    try:
        a = d.timestamp()
        return a
    except:
        return None

In [348]:
gamedates = games_df.gameDate.copy()
gamedates = gamedates.apply(timestamper)

In [349]:
games_df.gameDate = gamedates.fillna(method='bfill').apply(lambda x: datetime.datetime.fromtimestamp(x))

In [350]:
games_df.gameDate.isnull().sum()

0

In [351]:
games_df.head()

,leagueId,leagueLabel,year,stageTournament,stageRound,gameId,gameNumber,esportsPlatformId,blue,red,winner,gameScore,gameDate
0,98767991302996016,LEC,2020,LEC Spring 2020,Regular Season,103462460596524805,1,ESPORTSTMNT05:1410760,101383793569248484,101383793567806688,blue,-9.881120,2020-02-21 20:53:08.790
1,98767991302996016,LEC,2020,LEC Spring 2020,Regular Season,103462460596524807,1,ESPORTSTMNT05:1410131,101383793572656373,101383793567806688,blue,-6.101726,2020-02-14 19:11:25.807
2,98767991302996016,LEC,2020,LEC Spring 2020,Regular Season,103462460596524809,1,ESPORTSTMNT05:1400210,101383793574360315,101383793567806688,blue,-11.486386,2020-02-15 18:08:41.203
3,98767991302996016,LEC,2020,LEC Spring 2020,Regular Season,103462460596524811,1,ESPORTSTMNT04:1120607,103461966965149786,101383793567806688,blue,-13.574364,2020-02-07 18:04:21.154
4,98767991302996016,LEC,2020,LEC Spring 2020,Regular Season,103462460596524813,1,ESPORTSTMNT04:1130210,101383793567806688,98767991866488695,red,9.143119,2020-02-01 20:13:42.804


In [378]:
games_df['leagueId'].dtypes

dtype('int64')

## Bring LPL data from Oracle's Elixir for 2020, 2021, 2022 and 2023 spring

In [370]:
oe_df = pd.read_csv('lpl_outcomes.csv',sep=';')

In [371]:
oe_df.gameDate = pd.to_datetime(oe_df.gameDate,format='mixed')

In [372]:
oe_df = oe_df[(oe_df.split != 'Summer') | (oe_df.year != 2023)].copy()

In [373]:
oe_df.columns

Index(['esportsPlatformId', 'gameNumber', 'split', 'playoffs', 'year',
       'BlueTeamName', 'BlueTeamID', 'RedTeamName', 'RedTeamID', 'leagueLabel',
       'team_code_blue', 'team_code_red', 'winner', 'gameDate', 'gameScore'],
      dtype='object')

In [374]:
oe_df.rename(columns={'BlueTeamID':'blue','RedTeamID':'red'},inplace=True)

In [375]:
oe_df.columns

Index(['esportsPlatformId', 'gameNumber', 'split', 'playoffs', 'year',
       'BlueTeamName', 'blue', 'RedTeamName', 'red', 'leagueLabel',
       'team_code_blue', 'team_code_red', 'winner', 'gameDate', 'gameScore'],
      dtype='object')

In [376]:
oe_near_final_df = oe_df.drop(columns=['BlueTeamName','RedTeamName','team_code_blue','team_code_red'])

In [383]:
oe_near_final_df.rename(columns={'split':'stageTournament','playoffs':'stageRound'},inplace=True)

In [384]:
oe_near_final_df['leagueId']=98767991314006704

## Unify the datasets and prime them for elo_calculation.py

In [388]:
a_df = pd.concat([games_df,oe_near_final_df],axis=0).sort_values(by='gameDate',ascending=True)

We introduce K-score values for our tournaments here. Also a force multiplier for playoff games.

In [398]:
k_score_leagues = {
    'LEC': 40,
    'LCS': 40,
    'CBLOL': 30,
    'LCO': 30,
    'TCL': 30,
    'LCK': 40,
    'LJL': 30,
    'LLA': 30,
    'EMEA Masters': 20,
    'PCS': 30,
    'Worlds': 60,
    'LCL': 30,
    'MSI': 50,
    'VCS': 30,
    'LCK Academy': 20,
    'LPL': 40
}

In [399]:
a_df['kScore'] = a_df.leagueLabel.apply(lambda x: k_score_leagues.get(x,10))
a_df['kMult'] = a_df.stageRound.apply(lambda x: 1.2 if x == 'Playoffs' else 1)

a_df

,leagueId,leagueLabel,year,stageTournament,stageRound,gameId,gameNumber,esportsPlatformId,blue,red,winner,gameScore,gameDate,kScore,kMult
0,98767991314006704,LPL,2020,Spring,Regular Season,NaN,1.0,5655-7249,99566404848691211,99566404855553726,blue,-1.065438,2020-01-13 09:22:00.000,30,1.0
1,98767991314006704,LPL,2020,Spring,Regular Season,NaN,2.0,5655-7250,99566404848691211,99566404855553726,red,9.705381,2020-01-13 10:37:00.000,30,1.0
2,98767991314006704,LPL,2020,Spring,Regular Season,NaN,1.0,5656-7252,99566404846951820,99566404850008779,blue,-0.490556,2020-01-13 11:26:00.000,30,1.0
3,98767991314006704,LPL,2020,Spring,Regular Season,NaN,3.0,5655-7251,99566404848691211,99566404855553726,blue,-4.923935,2020-01-13 11:31:00.000,30,1.0
4,98767991314006704,LPL,2020,Spring,Regular Season,NaN,2.0,5656-7253,99566404850008779,99566404846951820,blue,-12.471972,2020-01-13 12:29:00.000,30,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20497,100695891328981120,EMEA Masters,2023,Summer 2023,Playoffs,110535609417095407,1.0,ESPORTSTMNT05:3239540,105536918475957734,105593951536470384,blue,-7.745563,2023-08-28 17:01:15.799,20,1.2
20501,100695891328981120,EMEA Masters,2023,Summer 2023,Playoffs,110535609417095414,2.0,ESPORTSTMNT05:3239544,107450819235010833,103877625775457850,red,0.305869,2023-08-28 17:46:41.999,20,1.2
20498,100695891328981120,EMEA Masters,2023,Summer 2023,Playoffs,110535609417095408,2.0,ESPORTSTMNT05:3240571,105536918475957734,105593951536470384,blue,-9.634254,2023-08-28 17:55:34.408,20,1.2
20502,100695891328981120,EMEA Masters,2023,Summer 2023,Playoffs,110535609417095415,3.0,ESPORTSTMNT05:3240572,103877625775457850,107450819235010833,blue,-13.758040,2023-08-28 18:40:41.605,20,1.2


In [403]:
a_df.loc[(a_df.leagueLabel=='TCL') & (a_df.year=='2023'),'kScore'] = 10

In [404]:
a_df.blue = a_df.blue.astype('int64')
a_df.red = a_df.red.astype('int64')

a_df.to_csv('process_everything.csv',sep=";",index=None)